# SVM

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import librosa
import librosa.display
import os
import cPickle

### Loading Train and validation data of gunshots and background

In [0]:
gun_train=np.zeros((5594,880))
gun_test=np.zeros((2400,880))
back_train=np.zeros((5594,880))
back_test=np.zeros((2400,880))
ind=0
gun_train_s="E:\\All Data\study\\MS\\2\\machine learning\\Project\\MIVIA_dataset\\MIVIA_DB4_dist\\training\\train_gunshots"
gun_test_s="E:\\All Data\study\\MS\\2\\machine learning\\Project\\MIVIA_dataset\\MIVIA_DB4_dist\\training\\train_gunshots"
back_train_s="E:\\All Data\\study\\MS\\2\\machine learning\\Project\\background_dataset\\train"
back_test_s="E:\\All Data\\study\\MS\\2\\machine learning\\Project\\background_dataset\\test"
path=gun_train_s
hop_length=512
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
    S=np.reshape(S,np.product(S.shape))
    gun_train[ind,:]=S
    ind=ind+1
    if ind>=5594:
        break
print "done loading gun_train"
path=gun_test_s
ind=0
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
#    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=20,fmax=8000)
#    S=librosa.feature.mfcc(S=librosa.power_to_db(S))
    S=np.reshape(S,np.product(S.shape))
    gun_test[ind,:]=S
    ind=ind+1
    if ind>=2400:
        break
print "done loading gun_test"
path=back_train_s
ind=0
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
#    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=20,fmax=8000)
#    S=librosa.feature.mfcc(S=librosa.power_to_db(S))
    S=np.reshape(S,np.product(S.shape))
    if S.shape[0]<880:    
        back_train[ind,0:S.shape[0]]=S[0:S.shape[0]]
    else:
        back_train[ind,:]=S[0:880]
    ind=ind+1
    if ind>=5594:
        break
print "done loading back_train"
path=back_test_s
ind=0
for i in os.listdir(path):
    y, sr = librosa.load(path+"\\"+i,duration=1)
    S = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=20)
#    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=20,fmax=8000)
#    S=librosa.feature.mfcc(S=librosa.power_to_db(S))
    S=np.reshape(S,np.product(S.shape))
    if S.shape[0]<880:    
        back_test[ind,:]=np.concatenate((S[0:860],S[0:20]))
    else:
        back_test[ind,:]=S[0:880]
    ind=ind+1
    if ind>=2400:
        break
print "done loading back_test"


done loading gun_train
done loading gun_test
done loading back_train
done loading back_test


#### Setting data for training and validation

In [0]:
train_X4=gun_train
test_X4=gun_test
train_y4=np.zeros((len(gun_train)))
test_y4=np.zeros((len(gun_test)))

train_X9=back_train
test_X9=back_test
train_y9=np.ones((len(back_train)))
test_y9=np.ones((len(back_test)))

train_set=np.concatenate((train_X4,train_X9),axis=0)
test_set=np.concatenate((test_X4,test_X9),axis=0)


train_y=np.concatenate((train_y4,train_y9),axis=0)
test_y=np.concatenate((test_y4,test_y9),axis=0)

X=np.concatenate((train_set,test_set))
y=np.concatenate((train_y,test_y))


print(train_set.shape,test_set.shape,train_y.shape,test_y.shape)
print(len(train_X4),len(test_X4),len(train_X9),len(test_X9))



((11188L, 880L), (4800L, 880L), (11188L,), (4800L,))
(5594, 2400, 5594, 2400)


###  Training with RBF kernel

In [0]:
error=[]
c=[]
error=np.zeros((4,10))
gama=[0.5,1,1.5,2]
for j in range(4):
    for i in range(10):
        c.append(1e-5*10**i)
        clf3= svm.SVC(C=c[i],kernel='rbf',gamma=gama[j])
        clf3.fit(train_set,train_y)
        Pe = 1 - clf3.score(test_set,test_y)
        error[j,i]=Pe

print("..........error..........")
print error
#min=np.where(error==np.argmin(error))
#min=np.reshape(min,(len(min)))
min = error.argmin()//error.shape[0], error.argmin()%error.shape[1]
print ("\n\n for C=%f and gamma=%f got minimum test_error=%f"%(c[min[1]],gama[min[0]],error[min[0],min[1]]))
print ("C=%f and gamma=%f was selected to retrain the classifier"%(c[min[1]],gama[min[0]]))

#retraining on full dataset for 
clf3= svm.SVC(C=c[min[1]],kernel='rbf',gamma=gama[min[0]])
clf3.fit(X,y)
Pe = 1 - clf3.score(test_set,test_y)
print ("\n\n for C=%f got minimum test_error=%f after retraining"%(c[min[1]],Pe))



..........error..........
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


 for C=0.000010 and gamma=0.500000 got minimum test_error=0.000000
C=0.000010 and gamma=0.500000 was selected to retrain the classifier


 for C=0.000010 got minimum test_error=0.000000 after retraining


In [0]:
print "saving trained model"
save_training = open('E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training_3.pickle','wb')
cPickle.dump(clf3,save_training,protocol=1)  # SAVE TRAINED CLASSIFIER
save_training.close()
print "loading trained model"
load_training = open("E:\\All Data\study\\MS\\2\\machine learning\\Project\\save_training_3.pickle",'rb')
new_clf = cPickle.load(load_training) # LOAD TRAINED CLASSIFIER
load_training.close()


#getting support vectors number and indices
support=new_clf.n_support_
print ("number of support_vectors",sum(support))

saving trained model
loading trained model
('number of support_vectors', 15988)
